# Running a CAT based on a synthetic correct/incorrect 3PL IRT model

This example shows how to run a CAT based on a synthetic correct/incorrect 3PL
IRT model.

Import order is important. We put ComputerAdaptiveTesting last so we get the
extra dependencies.

In [1]:
using Makie
import Pkg
import Random
using Distributions: Normal, cdf
using AlgebraOfGraphics
using ComputerAdaptiveTesting
using ComputerAdaptiveTesting.ExtraDistributions: NormalScaledLogistic
using ComputerAdaptiveTesting.Sim: auto_responder
using ComputerAdaptiveTesting.NextItemRules: AbilityVarianceStateCriterion
using ComputerAdaptiveTesting.TerminationConditions: FixedItemsTerminationCondition
using ComputerAdaptiveTesting.Aggregators: PriorAbilityEstimator, MeanAbilityEstimator, LikelihoodAbilityEstimator
using ComputerAdaptiveTesting.ItemBanks
using ComputerAdaptiveTesting.Integrators
import ComputerAdaptiveTesting.IntegralCoeffs
using CATPlots

@automakie()

Now we are read to generate our synthetic data using the supplied DummyData
module. We generate an item bank with 100 items and fake responses for 3
testees.

In [2]:
using ComputerAdaptiveTesting.DummyData: dummy_3pl, std_normal
Random.seed!(42)
(item_bank, question_labels, abilities, responses) = dummy_3pl(;num_questions=100, num_testees=3)

(ComputerAdaptiveTesting.ItemBanks.GuessItemBank{ComputerAdaptiveTesting.ItemBanks.TransferItemBank{Distributions.Logistic{Float64}}}([0.0, 0.0, 0.0, 0.08460030785740476, 0.4314519226411081, 0.3039209952022871, 0.0, 0.04492746298982733, 0.0, 0.043287010267455094  …  0.15254677941470662, 0.30746942855864307, 0.07918765671742968, 0.1074774854247907, 0.31567437317234404, 0.30719576934763493, 0.0, 0.0, 0.0, 0.0], ComputerAdaptiveTesting.ItemBanks.TransferItemBank{Distributions.Logistic{Float64}}(Distributions.Logistic{Float64}(μ=0.0, θ=0.5875440658049353), [0.7883556016042917, -0.8798585959543993, -0.873793482209626, -0.7332549644348927, -0.7021951987576804, -0.07258186804586991, -0.9129233863399265, 0.6316208311167526, 1.4386832757114134, -0.35416984337044605  …  -0.5445681210073285, 0.2208132624522603, 0.6006983743543721, 0.004286509104495785, 1.0262706957111953, -1.1165840371172657, -0.3013720710831042, 0.22759237809233804, 2.2573510919898294, 0.5003066411786888], [0.8658872814671111, 1

Simulate a CAT for each testee and record it using CatRecorder.
CatRecorder collects information which can be used to draw different types of plots.

In [3]:
const max_questions = 99
const integrator = FixedGKIntegrator(-6, 6, 80)
const dist_ability_est = PriorAbilityEstimator(std_normal)
const ability_estimator = MeanAbilityEstimator(dist_ability_est, integrator)
const rules = CatRules(
    ability_estimator,
    AbilityVarianceStateCriterion(dist_ability_est, integrator),
    FixedItemsTerminationCondition(max_questions)
)

const points = 500
xs = range(-2.5, 2.5, length=points)
raw_estimator = LikelihoodAbilityEstimator()
recorder = CatRecorder(xs, responses, integrator, raw_estimator, ability_estimator)
for testee_idx in axes(responses, 2)
    tracked_responses, θ = run_cat(
        CatLoopConfig(
            rules=rules,
            get_response=auto_responder(@view responses[:, testee_idx]),
            new_response_callback=(tracked_responses, terminating) -> recorder(tracked_responses, testee_idx, terminating),
        ),
        item_bank
    )
    true_θ = abilities[testee_idx]
    abs_err = abs(θ - true_θ)
end

Make a plot showing how the estimated value evolves during the CAT.
We also plot the 'true' values used to generate the responses.

In [4]:
conv_lines_fig = ability_evolution_lines(recorder; abilities=abilities)
conv_lines_fig

FigureGrid()

Make an interactive plot, showing how the distribution of the ability
likelihood evolves.

In [5]:
conv_dist_fig = lh_evoluation_interactive(recorder; abilities=abilities)
conv_dist_fig

Figure()

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*